# Multiplying upper triangular matrices

This notebook helps you implement the operation $ C := U R $ where $ C, U, R \in \mathbb{R}^{n \times n} $, and $ U $ and $ R $ are upper triangular.  $ U $ and $ R $ are stored in the upper triangular part of numpy matrices <code> U </code> and <code> R </code>. The upper triangular part of matrix <code> C </code> is to be overwritten with the resulting upper triangular matrix.  

First, we create some matrices.

In [28]:
n = 5

C = rand(n, n)
println( "C = " )
C

C = 


5×5 Array{Float64,2}:
 0.342006  0.0125096  0.973661  0.117183  0.452058
 0.305406  0.1648     0.342776  0.221853  0.674224
 0.218045  0.721887   0.715148  0.684214  0.162473
 0.509953  0.538232   0.251733  0.131354  0.882886
 0.127258  0.190803   0.068036  0.289312  0.604951

In [29]:
Cold = copy( C ) # an alternative way of doing a "hard" copy, in this case of a matrix
    
U = rand(n, n)
println( "U = " )
U

U = 


5×5 Array{Float64,2}:
 0.92718   0.510549  0.517715  0.397911  0.777794
 0.954084  0.358228  0.425726  0.475012  0.809655
 0.742692  0.288332  0.892767  0.300795  0.397479
 0.718162  0.25261   0.939957  0.676838  0.738155
 0.946664  0.569021  0.82884   0.151853  0.323365

In [30]:
R = rand(n, n)
println( "R = " )
R

R = 


5×5 Array{Float64,2}:
 0.16368   0.0327048  0.2613    0.588521  0.507738
 0.111698  0.861812   0.871939  0.460539  0.214001
 0.113055  0.113623   0.977569  0.750129  0.616447
 0.127917  0.271882   0.807505  0.671459  0.519208
 0.15232   0.393337   0.682602  0.862563  0.181089

## <h2>The algorithm  </h2>  

<img src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/5_5_1_10_Answer.png" alt="Upper triangular matrix-matrix multiplication" width="80%">

<h2> The routine <code> Trtrmm_uu_unb_var1!( U, R, C ) </code> </h2>

This routine computes $ C := U R + C $.  The "\_uu\_" means that $ U $ and $ R $ are upper triangular matrices (which means $ C $ is too).  However, the lower triangular parts of numpy matrices <code> U </code>, <code> R </code>, and <code> C </code> are not to be "touched".
    
The specific laff function you will want to use is some subset of
<ul>
<li> <code> laff.gemv!( trans, alpha, A, x, beta, y ) </code> which computes $ y := \alpha A x + \beta y $ or $ y := \alpha A^T x + \beta y $ depending on whether <code> trans = 'No transpose' </code> or <code> trans = 'Transpose' </code> </li>
    <li> <code> laff.ger!( alpha, x, y, A ) </code> which computes the rank-1 update (adds a multiple of an outer product to a matrix)
$ A := \alpha x y^T + A $. </li>
    <li> <code> laff.axpy!( alpha, x, y ) </code></li>
    <li>    <code> laff.dots!( x, y, alpha ) </code></li>
</ul>

<font color=red> Hint:</font>  If you multiply with $ U_{00} $, you will want to use <code> Matrix(UpperTriangular( U00 )) </code>; `UpperTriangular(U00)` will make sure you don't compute with the nonzeroes below the diagonal and wrapping this in a call to `Matrix()` will ensure you're still passing a regular `Matrix` type to the relevant `laff` function`.

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [31]:
using LinearAlgebra

In [37]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Trtrmm_uu_unb_var1!(U, R, C)

    UTL, UTR, 
    UBL, UBR  = flame.part_2x2(U, 
                               0, 0, "TL")

    RTL, RTR, 
    RBL, RBR  = flame.part_2x2(R, 
                               0, 0, "TL")

    CTL, CTR, 
    CBL, CBR  = flame.part_2x2(C, 
                               0, 0, "TL")

    while size(UTL, 1) < size(U, 1)

        U00,  u01,       U02,  
        u10t, upsilon11, u12t, 
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, 
                                                         UBL, UBR, 
                                                         1, 1, "BR")

        R00,  r01,   R02,  
        r10t, rho11, r12t, 
        R20,  r21,   R22   = flame.repart_2x2_to_3x3(RTL, RTR, 
                                                     RBL, RBR, 
                                                     1, 1, "BR")

        C00,  c01,     C02,  
        c10t, gamma11, c12t, 
        C20,  c21,     C22   = flame.repart_2x2_to_3x3(CTL, CTR, 
                                                       CBL, CBR, 
                                                       1, 1, "BR")

        #------------------------------------------------------------#

        # imporant:  You do not need to recompute C00 = U00 * R00!!!!
        laff.gemv!( "No transpose", 1.0, Matrix(UpperTriangular(U00)), r01, 0.0, c01 )
        laff.axpy!( rho11, u01, c01 )
        laff.dots!( rho11, upsilon11, gamma11 )

        #------------------------------------------------------------#

        UTL, UTR, 
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  
                                               u10t, upsilon11, u12t, 
                                               U20,  u21,       U22,  
                                               "TL")

        RTL, RTR, 
        RBL, RBR  = flame.cont_with_3x3_to_2x2(R00,  r01,   R02,  
                                               r10t, rho11, r12t, 
                                               R20,  r21,   R22,  
                                               "TL")

        CTL, CTR, 
        CBL, CBR  = flame.cont_with_3x3_to_2x2(C00,  c01,     C02,  
                                               c10t, gamma11, c12t, 
                                               C20,  c21,     C22,  
                                               "TL")

    end

    flame.merge_2x2!(CTL, CTR, 
                     CBL, CBR, C)

end

Trtrmm_uu_unb_var1! (generic function with 1 method)

In [38]:
C = copy( Cold ) # restore C 

Trtrmm_uu_unb_var1!( U, R, C )

using LinearAlgebra
Cref = UpperTriangular( U ) * UpperTriangular( R ) + LowerTriangular(Cold)
println( "C - Cref = " )
C - Cref

C - Cref = 


5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0        
 0.0  0.0  0.0  0.0  0.0        
 0.0  0.0  0.0  0.0  1.11022e-16
 0.0  0.0  0.0  0.0  0.0        
 0.0  0.0  0.0  0.0  0.0        

In theory, the result matrix should be (approximately) zero.

## Watch the algorithm at work!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.